In [ ]:
!pip install "opencv-python-headless<4.3"
!pip install cellpose

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.10.0.84
    Uninstalling opencv-python-headless-4.10.0.84:
      Successfully uninstalled opencv-python-headless-4.10.0.84
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.3.1 requires opencv-python-headless>=4.1.1, but you have opencv-python-headless 3.4.18.65 which is incompatible.
qudida 0.0.4 requires opencv-python-headless>=4.0.1, but you have opencv-python-headless 3.4.18.65 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.0/205.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 11.7 MB/s eta 0:00:00
  Using cached nvidia_

In [ ]:
from cellpose import core, utils, io, models, metrics
use_GPU = core.use_gpu()

In [ ]:

from cellpose import io

io.logger_setup() # run this to get printing of progress

# DEFINE CELLPOSE MODEL WITH SIZE
# model_type='cyto3' or model_type='nuclei'
model = models.Cellpose(gpu=True, model_type="cyto3")

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus
# OR if you have different types of channels in each image
# channels = [[2,3], [0,0], [0,0]]

# FOR A CUSTOM MODEL OR OTHER BUILT-IN MODEL:
# model = models.CellposeModel(gpu=True, model_type="livecell_cp3")

creating new log file


100%|██████████| 25.3M/25.3M [00:00<00:00, 92.1MB/s]
100%|██████████| 3.54k/3.54k [00:00<00:00, 4.91MB/s]


In [ ]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 200
from cellpose import utils, io, transforms
import random
import PIL

In [ ]:
# prompt: Read this spreadsheet into a pandas dataframe 1FNRYtK4Tqfgf3AL8ntO-imfdXVnf7K5c

from google.colab import auth
auth.authenticate_user()
# import the gspread library
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
data_sheet = '<path to data>'
sheet = gc.open_by_url(data_sheet).get_worksheet(0)

# prompt: turn sheet above into a dataframe. First row as header

import pandas as pd
rows = sheet.get_all_values()
df = pd.DataFrame.from_records(rows[1:], columns=rows[0])

In [ ]:
# Filter just the Organoid Group normal
df = df[df['Organoid Group'] == 'Tumor']
control_df = df[df['Control / Treatment?'] == 'Control']
treatment_df = df[df['Control / Treatment?'] == 'Treatment']

In [ ]:
data_dir = '/path/to/data'
analysis_dir = '/path/to/analysis'

In [ ]:
all_files = []
import os
for root, dirs, files in os.walk(data_dir):
  for fn in files:
    if fn.endswith(".tif"):
      all_files.append(os.path.join(root, fn))


In [ ]:
file2path = {x.split('/')[-1]:x for x in all_files}

In [ ]:
DAPI = '_DAPI'
PHAL = '_PHAL'
ECAD = '_ECAD'
PHAL_DAPI = '_PHAL+DAPI'
ECAD_DAPI = '_ECAD+DAPI'
def get_image(name, suffix):
  filename = name+suffix
  return file2path.get(filename+'.tif')

In [ ]:
def get_masks(df):
  imgs = []
  names = []
  for name in df[' Image Series name (Initial)']:
    fn = get_image(name, PHAL_DAPI)
    if fn is None:
      continue
    names.append(name)
    img = PIL.Image.open(fn)
    img = np.array(img)
    # resize to 400x400
    if img.shape[0] > 400 or img.shape[1] > 400:
      img = transforms.resize_image(img, Lx=400, Ly=400)
    # Change to C, X, Y
    if img.shape[2] == 3:
      img = img.transpose(2,0,1)
    imgs.append(img)

  masks_pred, flows, styles, diams = model.eval(imgs, diameter=0, channels=[2,3],
                                                niter=2000) # using more iterations for bacteria
  name_to_mask = {}
  for name, mask in zip(names, masks_pred):
    name_to_mask[name] = mask
  return name_to_mask

In [ ]:
control_masks = get_masks(control_df)
treatment_masks = get_masks(treatment_df)

In [ ]:
import pickle

with open(analysis_dir + '/control_masks.pkl', 'wb') as f:
  pickle.dump(control_masks, f)
with open(analysis_dir + '/treatment_masks.pkl', 'wb') as f:
  pickle.dump(treatment_masks, f)